In [83]:
from PIL import Image
import os, sys
import cv2
import numpy as np
from imutils.object_detection import non_max_suppression
import random

In [84]:
random.seed(42)

#### You can change some hyperparameters here

In [85]:
# some hyperparameter
frame_interval = 25
east_confidence = 0.5
opflow_min_area_threshold = 700
opflow_height_increase = 0
num_frames_for_overlap_check = 3
rolling_text_confidence = 0.2 # if the overlapping area / east box area > 0.2, this is a part of the rolling text

#### You can change the video path here

In [86]:
# file path
vidpath = '../data/train4_crop.mp4'

### Extract frames

In [87]:
def files(vidpath):
    base_name = os.path.basename(vidpath)
    vidname, _ = os.path.splitext(base_name)  # Split the base name to get the name without the extension
    
    try: # Remove image frames path if it already exists
        os.remove(f"{vidname}_image_frames")
    except OSError:
        pass
    if not os.path.exists(f"{vidname}_image_frames"): # create the directory if it does not already exist
        os.makedirs(f"{vidname}_image_frames")

    src_vid = cv2.VideoCapture(vidpath) # open the video file
    return(src_vid)
    
def process(src_vid, frame_interval):
    base_name = os.path.basename(vidpath)
    vidname, _ = os.path.splitext(base_name)  # Split the base name to get the name without the extension
    saved_frame_nums = []
    
    index = 0
    while src_vid.isOpened():
        ret, frame = src_vid.read() 
        if not ret: # break at the end of the video (ret returns True if frame is succesfully read)
            break
        name = f'./{vidname}_image_frames/frame{str(index)}.png'

        if index % frame_interval == 0: # every 50th frame will be saved: can adjust this number to capture more or less frames
            print('Extracting frame...' + name)
            cv2.imwrite(name, frame)
            saved_frame_nums.append(index)
        index = index + 1
        
    src_vid.release()
    cv2.destroyAllWindows()
    
    video_frames_dir = f"./{vidname}_image_frames"
    
    return saved_frame_nums, video_frames_dir

In [88]:
vid = files(vidpath)
saved_frame_nums, video_frames_dir = process(vid, frame_interval)

Extracting frame..../train4_crop_image_frames/frame0.png
Extracting frame..../train4_crop_image_frames/frame25.png
Extracting frame..../train4_crop_image_frames/frame50.png
Extracting frame..../train4_crop_image_frames/frame75.png
Extracting frame..../train4_crop_image_frames/frame100.png
Extracting frame..../train4_crop_image_frames/frame125.png
Extracting frame..../train4_crop_image_frames/frame150.png
Extracting frame..../train4_crop_image_frames/frame175.png
Extracting frame..../train4_crop_image_frames/frame200.png
Extracting frame..../train4_crop_image_frames/frame225.png
Extracting frame..../train4_crop_image_frames/frame250.png
Extracting frame..../train4_crop_image_frames/frame275.png
Extracting frame..../train4_crop_image_frames/frame300.png
Extracting frame..../train4_crop_image_frames/frame325.png
Extracting frame..../train4_crop_image_frames/frame350.png
Extracting frame..../train4_crop_image_frames/frame375.png
Extracting frame..../train4_crop_image_frames/frame400.png
Ex

### Optical Flow

In [89]:
def draw_horizontal_flow(img, flow, step=30):
    """Draw optical flow vectors on the video."""
    h, w = img.shape[:2]
    y, x = np.mgrid[step//2:h:step, step//2:w:step].reshape(2, -1).astype(int)
    fx = flow[y, x, 0]
    fy = flow[y, x, 1]

    # Keep only leftward motion vectors (negative horizontal flow)
    fx[fx > 0] = 0

    lines = np.vstack([x, y, x + fx, y + fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    for (x1, y1), (x2, y2) in lines:
        cv2.arrowedLine(vis, (x1, y1), (x2, y2), (0, 255, 0), 1, tipLength=0.5)
    return vis

def detect_and_save_screengrab(video_path):
    cap = cv2.VideoCapture(video_path)
    
    # Get total number of frames and calculate halfway point
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    halfway_frame = total_frames // 2

    ret, first_frame = cap.read()
    if not ret:
        print("Failed to read the video file.")
        return

    prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

    # Initialize a mask for consistent leftward movement
    consistent_motion_mask = None
    consistency_threshold = 0.8  # Consistent movement threshold (percentage of frames)
    num_frames = 0

    saved_screengrab = False
    screengrab_frame = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 5, 15, 3, 5, 1.2, 0)
        
        # Calculate the mask of leftward movement
        leftward_motion_mask = flow[..., 0] < 0
        
        # Initialize the consistent motion mask
        if consistent_motion_mask is None:
            consistent_motion_mask = np.zeros_like(leftward_motion_mask, dtype=np.float32)
        
        # Update the consistent leftward movement mask
        consistent_motion_mask[leftward_motion_mask] += 1
        num_frames += 1

        # Calculate the ratio of consistent leftward movement
        consistent_ratio = consistent_motion_mask / num_frames
        consistent_leftward_regions = consistent_ratio > consistency_threshold

        # Apply morphological operations to fill in the gaps
        consistent_leftward_mask = consistent_leftward_regions.astype(np.uint8)
        kernel = np.ones((5, 5), np.uint8)
        consistent_leftward_mask = cv2.morphologyEx(consistent_leftward_mask, cv2.MORPH_CLOSE, kernel)

        # Create an image for highlighting
        consistent_leftward_img = np.zeros_like(frame)
        consistent_leftward_img[consistent_leftward_mask == 1] = [0, 255, 0]  # Green

        vis = draw_horizontal_flow(gray, flow)

        # Combine visualization with consistent leftward movement highlight
        combined_vis = cv2.addWeighted(vis, 0.7, consistent_leftward_img, 0.3, 0)

        # Save screengrab at halfway point
        if num_frames == halfway_frame and not saved_screengrab:
            screengrab_frame = combined_vis.copy()
            saved_screengrab = True
        
        prev_gray = gray.copy()

    # Save the screengrab outside the loop to avoid delay
    if screengrab_frame is not None:
        vidname = str(video_path)
        cv2.imwrite(f'{vidname}_screengrab_halfway.png', screengrab_frame)
    
    cap.release()
    cv2.destroyAllWindows()

In [90]:
########## CALL TO FUNCTION:
detect_and_save_screengrab(vidpath)

#### Draw a bounding box around the detected regions

In [91]:
def extract_bounding_boxes(vidpath, height_increase=0, opflow_min_area_threshold=700):
    opflow_box_coordinates = []
    screengrab_path = f'{vidpath}_screengrab_halfway.png'
    
    screengrab = cv2.imread(screengrab_path)
    if screengrab is None:
        print("Failed to read the screengrab.")
        return

    # Convert to HSV and create a mask for the green color
    hsv = cv2.cvtColor(screengrab, cv2.COLOR_BGR2HSV)
    lower_green = np.array([40, 40, 40])
    upper_green = np.array([80, 255, 255])
    mask = cv2.inRange(hsv, lower_green, upper_green)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw bounding boxes around significant motion regions
    for contour in contours:
        if cv2.contourArea(contour) > opflow_min_area_threshold:  # Adjust the threshold for minimum area as needed
            x, y, w, h = cv2.boundingRect(contour)
            # Increase the height of the bounding box
            y = max(0, y - height_increase // 2)
            h = h + height_increase
            # Ensure the bounding box is within the image boundaries
            h = min(h, screengrab.shape[0] - y)
            cv2.rectangle(screengrab, (x, y), (x + w, y + h), (0, 255, 0), 2)
            opflow_box_coordinates += [[x, y, x+w, y+h]]

    # Save and display the image with bounding boxes
    cv2.imwrite(f'{vidpath}_screengrab_with_bounding_boxes.png', screengrab)
    
    return opflow_box_coordinates

########## CALL TO FUNCTION:
opflow_box_coordinates = extract_bounding_boxes(vidpath, opflow_height_increase, opflow_min_area_threshold) # Add extra height to ensure no text is cut off from the top/bottom

### EAST for text detection

In [92]:
def east_detect(image, output_path, east_confidence):
    ''' uses EAST detection to get x,y coordinates of all bounding boxes of input image 
    
    param images: paths to the image
    param output_path: path where to save copy of original image with bounding boxes drawn on top
    
    returns list of tuples: each tuple contains (x_min, y_min, x_max, y_max) of each detected bounding box

    code from: https://medium.com/technovators/scene-text-detection-in-python-with-east-and-craft-cbe03dda35d5
    '''
    
    
    layerNames = [
    	"feature_fusion/Conv_7/Sigmoid",
    	"feature_fusion/concat_3"]
    
    orig = image.copy()
    
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    
    (H, W) = image.shape[:2]
    
    # set the new width and height and then determine the ratio in change
    # for both the width and height: Should be multiple of 32
    (newW, newH) = (672, 672)
    
    rW = W / float(newW)
    rH = H / float(newH)
    
    # resize the image and grab the new image dimensions
    image = cv2.resize(image, (newW, newH))
    
    (H, W) = image.shape[:2]
    
    net = cv2.dnn.readNet("frozen_east_text_detection.pb")
    
    blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
    	(123.68, 116.78, 103.94), swapRB=True, crop=False)
    
    net.setInput(blob)
    
    (scores, geometry) = net.forward(layerNames)
    
    (numRows, numCols) = scores.shape[2:4]
    rects = []
    confidences = []
    # loop over the number of rows
    for y in range(0, numRows):
        # extract the scores (probabilities), followed by the geometrical
        # data used to derive potential bounding box coordinates that
        # surround text
        scoresData = scores[0, 0, y]
        xData0 = geometry[0, 0, y]
        xData1 = geometry[0, 1, y]
        xData2 = geometry[0, 2, y]
        xData3 = geometry[0, 3, y]
        anglesData = geometry[0, 4, y]
    
        for x in range(0, numCols):
    		# if our score does not have sufficient probability, ignore it
            # Set minimum confidence as required
            if scoresData[x] < east_confidence:
                continue
    		# compute the offset factor as our resulting feature maps will
            #  x smaller than the input image
            (offsetX, offsetY) = (x * 4.0, y * 4.0)
            # extract the rotation angle for the prediction and then
            # compute the sin and cosine
            angle = anglesData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)
            # use the geometry volume to derive the width and height of
            # the bounding box
            h = xData0[x] + xData2[x]
            w = xData1[x] + xData3[x]
            # compute both the starting and ending (x, y)-coordinates for
            # the text prediction bounding box
            endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
            endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
            startX = int(endX - w)
            startY = int(endY - h)
            # add the bounding box coordinates and probability score to
            # our respective lists
            rects.append((startX, startY, endX, endY))
            confidences.append(scoresData[x])
                        
    boxes = non_max_suppression(np.array(rects), probs=confidences)
    # loop over the bounding boxes
    coordinates = []
    for (startX, startY, endX, endY) in boxes:
        # scale the bounding box coordinates based on the respective
        # ratios
        startX = int(startX * rW)
        startY = int(startY * rH)
        endX = int(endX * rW)
        endY = int(endY * rH)
        # draw the bounding box on the image
        # startX,startY ------
        # |                 |
        # |                 |
        # |                 |
        # ----------endX,endY
        out_image = cv2.rectangle(orig, (startX, startY), (endX, endY), (0, 255, 0), 2)
        # out_image = cv2.rectangle(out_image, (146, 888), (1920, 1002), (0, 255, 0), 2)
        coordinates += [[startX, startY, endX, endY]]
    
    cv2.imwrite(f"{output_path}", out_image)
    return coordinates


In [93]:
coordinates_list= []

for frame_number in saved_frame_nums:
	frame_path = video_frames_dir + f"/frame{frame_number}.png" # path to individual frame being processed

	base_name = os.path.basename(frame_path)
	frame_name, _ = os.path.splitext(base_name) # get just the individual frame name without the extension

	if not os.path.isdir(f"{video_frames_dir}/output"):
		os.makedirs(f"{video_frames_dir}/output") # makes output directory for the frame being processed
	
	image = cv2.imread(frame_path) 
	coordinates = east_detect(image, f"{video_frames_dir}/output/out_{frame_name}_EAST.png", east_confidence) # get coordinates of bounding boxes and save image to new output dir for that frame

	coordinates_list += [coordinates]


### Calculate the overlapping area of optic flow rectangles and east rectangles.

randomly get {N} frames to do this:
* For each opflow rectangle, get the overlapping area sum 
* The rectangle with the largest overlapping area -> rolling text region \
-> some sanity check for rectangles?

In [94]:
opflow_box_coordinates

[[1877, 754, 1920, 791], [696, 549, 1157, 592]]

In [95]:
coordinates_list

[[[820, 596, 1014, 630],
  [865, 718, 1077, 748],
  [1034, 589, 1102, 626],
  [548, 691, 637, 721],
  [942, 567, 1040, 586],
  [868, 683, 1011, 711],
  [537, 728, 620, 761],
  [717, 604, 797, 634],
  [697, 728, 765, 756],
  [1011, 679, 1085, 708],
  [557, 662, 654, 683],
  [725, 689, 820, 716],
  [611, 557, 682, 583],
  [1014, 564, 1085, 591],
  [837, 567, 968, 593],
  [565, 586, 645, 609],
  [725, 572, 820, 591],
  [1588, 898, 1660, 919],
  [1837, 585, 1900, 613],
  [754, 546, 860, 565]],
 [[814, 589, 1017, 625],
  [845, 713, 1057, 745],
  [848, 676, 988, 708],
  [1017, 583, 1094, 620],
  [531, 686, 622, 716],
  [805, 564, 945, 586],
  [697, 599, 777, 628],
  [531, 655, 640, 676],
  [954, 560, 1031, 583],
  [1105, 531, 1177, 552],
  [520, 723, 602, 761],
  [731, 565, 828, 586],
  [1011, 673, 1074, 705],
  [682, 721, 760, 752],
  [737, 681, 808, 711],
  [548, 581, 617, 604],
  [1034, 535, 1114, 552],
  [917, 98, 980, 120],
  [714, 541, 828, 560],
  [608, 554, 668, 576],
  [1645, 564, 1

In [96]:
def calculate_overlap_area(rect1, rect2):
    """
    Calculate the overlapping area between two rectangles.
    
    rect1 and rect2 are tuples in the form (startX, startY, endX, endY)
    """
    x1_max = max(rect1[0], rect2[0])
    y1_max = max(rect1[1], rect2[1])
    x2_min = min(rect1[2], rect2[2])
    y2_min = min(rect1[3], rect2[3])
    
    overlap_width = max(0, x2_min - x1_max)
    overlap_height = max(0, y2_min - y1_max)
    
    if x1_max >= x2_min or y1_max >= y2_min:
        return 0
    
    return overlap_width * overlap_height


def sum_overlapping_areas(main_rect, rectangles):
    """
    Calculate the sum of overlapping areas of multiple rectangles with the main rectangle.
    
    main_rect is a tuple in the form (startX, startY, endX, endY)
    rectangles is a list of tuples in the same form
    """
    total_overlap_area = 0
    for rect in rectangles:
        total_overlap_area += calculate_overlap_area(main_rect, rect)
    
    return total_overlap_area


In [97]:

overlap_areas = []
choose_frames = random.sample(list(range(len(saved_frame_nums))), num_frames_for_overlap_check)

for opflow_box_coordinate in opflow_box_coordinates:
    total_overlap = 0
    for frame_index in choose_frames:
        total_overlap += sum_overlapping_areas(opflow_box_coordinate, coordinates_list[frame_index])
    overlap_areas.append(total_overlap)

rolling_box_index = np.argmax(np.array(overlap_areas))
rolling_box = opflow_box_coordinates[rolling_box_index]

#### After getting the rolling box

* For all the east boxes in each frame, calculate their overlapping area with the rolling box
* If overlapping area / east box area >= rolling_text_confidence, this is a part of the rolling text
* Crop this part and save to the folder in each frame. \
TBD: The cropped text images should follow a left to right sequence

In [ ]:
# TODO: The cropped text images should follow a left to right sequence

In [98]:
def crop_and_save_image(image_path, coords, output_path):
    ''' saves crops of original image according to specified coordinates 

    param image_path: path to original image
    param coordinates: coordinates of bounding boxes in tuple (x_min, y_min, x_max, y_max)
    param output_path: path of output file'''
    
    img = cv2.imread(image_path)
    x_min, y_min, x_max, y_max = coords
    cropped_img = img[y_min:y_max, x_min:x_max]

    # Save
    cv2.imwrite(output_path, cropped_img)


In [107]:
for frame_num, rects in zip(saved_frame_nums, coordinates_list):
    frame_path = f"{video_frames_dir}/frame{frame_num}.png"
    output_path = f"{video_frames_dir}/croppedFrame{frame_num}"
    if os.path.isdir(output_path):
        print("The cropped folders are there! You can delete them :)")
        continue
    else:
        os.makedirs(output_path)
        
    for idx, rect in enumerate(rects):
        overlap_area = calculate_overlap_area(rolling_box, rect)
        rect_area = (rect[2] - rect[0]) * (rect[3] - rect[1])
        if rect_area > 0 and (overlap_area / rect_area) >= rolling_text_confidence:
            final_outout_path = f"{output_path}/{idx}.jpg"
            crop_and_save_image(frame_path, rect, final_outout_path)
            print(f"Saved cropped image to {final_outout_path} for rectangle {rect}")

Saved cropped image to ./train4_crop_image_frames/croppedFrame0/4.jpg for rectangle [942, 567, 1040, 586]
Saved cropped image to ./train4_crop_image_frames/croppedFrame0/13.jpg for rectangle [1014, 564, 1085, 591]
Saved cropped image to ./train4_crop_image_frames/croppedFrame0/14.jpg for rectangle [837, 567, 968, 593]
Saved cropped image to ./train4_crop_image_frames/croppedFrame0/16.jpg for rectangle [725, 572, 820, 591]
Saved cropped image to ./train4_crop_image_frames/croppedFrame0/19.jpg for rectangle [754, 546, 860, 565]
Saved cropped image to ./train4_crop_image_frames/croppedFrame25/5.jpg for rectangle [805, 564, 945, 586]
Saved cropped image to ./train4_crop_image_frames/croppedFrame25/8.jpg for rectangle [954, 560, 1031, 583]
Saved cropped image to ./train4_crop_image_frames/croppedFrame25/11.jpg for rectangle [731, 565, 828, 586]
Saved cropped image to ./train4_crop_image_frames/croppedFrame25/18.jpg for rectangle [714, 541, 828, 560]
Saved cropped image to ./train4_crop_imag